In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [5]:
df = pd.read_csv("/Users/gillestondeur/code/syeda-tabassum-rahaman/scam-job-detector/raw_data/fake_job_postings.csv")

In [6]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [7]:
df.function.nunique()

37

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [9]:
df.isnull().sum()


job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [10]:
df['fraudulent'].value_counts(normalize=True)

fraudulent
0    0.951566
1    0.048434
Name: proportion, dtype: float64

# proportion of fraudulent vs non-fraudulent job postings skwewed --> address to handle class imbalance

In [11]:
req =df.location.nunique()
req

3105

In [12]:
df.sample(100)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
4187,4188,"Video journalist, Urdu Language","US, DC, Washington",NaN,NaN,Applied Memetics LLC is a professional service...,The Video Journalist shall have at least five ...,The Video Journalist shall properly complete a...,NaN,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Broadcast Media,NaN,0
4616,4617,Associate Business Development,"NL, , Work from home",NaN,NaN,NaN,Want to build a career in IT? Free training in...,NaN,NaN,0,0,0,Full-time,Entry level,NaN,Information Technology and Services,Business Development,0
15734,15735,"Home Health Aid/CNA- Reserve, NM","US, NM, Reserve",NaN,NaN,"Critical Nurse Staffing, Inc. is enrolled with...","Critical Nurse Staffing, Inc. is seeking an HH...",NaN,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
14795,14796,Transaction Officers,"NG, LA,",NaN,NaN,The MacTay Group is a leading consulting compa...,"Our client, a major bank in the Nigerian econo...",Skills RequiredGood customer’s service careGoo...,NaN,0,1,0,Full-time,Entry level,Bachelor's Degree,Banking,Customer Service,0
12495,12496,Senior Area Sales Manager - Mumbai,"IN, MH, Mumbai",Sales,NaN,POSist is a completely cloud based POS for Res...,Job descriptionRoles &amp; Responsibilities:Sa...,Skills Requirements/ExpectationsExperience in ...,NaN,0,1,0,Full-time,Mid-Senior level,Unspecified,Information Technology and Services,Sales,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5284,5285,Senior Developer,"GB, LND, London",Core,NaN,"Social Media has the power to shape, educate a...",Forward Partners is a startup foundry that bui...,"About youHave full stack web experience, with ...",NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,0
17270,17271,Sales Executive / Engineer,"MX, DIF, México",NaN,NaN,Quiénes SomosSomos una firma mexicana de jóven...,"Sales Executive / Engineer | $10,000 – $15,00...","Key responsibilities:Establish, maintain and e...",Constant quality training.Exponential growth b...,0,1,1,Full-time,Associate,High School or equivalent,Marketing and Advertising,Sales,0
15950,15951,Manufacturing Engineering Manager,"US, MN, Minneapolis",NaN,NaN,We Provide Full Time Permanent Positions for m...,(We have more than 1500+ Job openings in our w...,NaN,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
16590,16591,Friendly Customer Service Employees Start IMME...,"US, KY, Lexington",NaN,NaN,"LEI Home Enhancements, is an Ohio based compan...",Seeking Experienced Customer Service Represent...,We're looking for candidates who are: * Discip...,We offer competitive pay; benefits; training/c...,0,1,0,NaN,NaN,NaN,NaN,Customer Service,0


In [13]:
temp = df.has_questions.value_counts()
temp

has_questions
0    9088
1    8792
Name: count, dtype: int64

In [14]:
X = df.drop(columns=['fraudulent'])
y = df['fraudulent']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# catagoreical columsn for One-Hot Encoding
categorical_columns = [
    'country',
    'industry',
    'function',
    'employment_type',
]
ordinal_columns = [
    'required_experience',
    'required_education'
]

In [32]:
required_education_values = pd.DataFrame(df.required_education.unique()).iloc[:,0].tolist()
required_experience_values = pd.DataFrame(df.required_experience.unique()).iloc[:,0].tolist()


In [33]:
required_experience_values

['Internship',
 'Not Applicable',
 nan,
 'Mid-Senior level',
 'Associate',
 'Entry level',
 'Executive',
 'Director']

In [29]:
required_education_values

[nan,
 "Bachelor's Degree",
 "Master's Degree",
 'High School or equivalent',
 'Unspecified',
 'Some College Coursework Completed',
 'Vocational',
 'Certification',
 'Associate Degree',
 'Professional',
 'Doctorate',
 'Some High School Coursework',
 'Vocational - Degree',
 'Vocational - HS Diploma']

In [34]:
experience_order = [
    "Not Applicable",
    "Unknown",
    "Internship",
    "Entry level",
    "Associate",
    "Mid-Senior level",
    "Director",
    "Executive"
]

education_order = [
    "Unknown",
    "High School or equivalent",
    "Vocational",
    "Certification",
    "Some College Coursework Completed",
    "Associate Degree",
    "Bachelor's Degree",
    "Professional",
    "Master's Degree"
]

In [37]:
cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)
ordinal_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(
    categories=[experience_order, education_order],
    handle_unknown="use_encoded_value",
    unknown_value=-1)
)


In [38]:
preprocessing = make_pipeline(
    cat_transformer,
    ordinal_transformer
)